# Выбор локации для скважины

Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.


**Выбор региона для разработки месторождений нефти:**

Исходя из данных о бурении в трех регионах, построим модель прогнозирующую прибыльность инвестиций в разработку месторождений конкретного региона, с учетом предоженных условий:

**План**

1. Изучим и подготовим данные. 
2. Обучим и проверим модель "линейная регрессия", отдельно для каждого региона.
3. Рассчитаем прогнозные прибыли и риски для каждого региона. 
4. Определим наилучший регион для инвестиций. 

## Загрузка и подготовка данных

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from scipy import stats as st

In [2]:

try:
    geo_0 = pd.read_csv('/datasets/geo_data_0.csv')
    geo_1 = pd.read_csv('/datasets/geo_data_1.csv')
    geo_2 = pd.read_csv('/datasets/geo_data_2.csv')
except:
    geo_0 = pd.read_csv('C:/ya_pr/geo_rig/geo_data_0.csv')
    geo_1 = pd.read_csv('C:/ya_pr/geo_rig/geo_data_1.csv')
    geo_2 = pd.read_csv('C:/ya_pr/geo_rig/geo_data_2.csv')



In [3]:
print(geo_0.head(10))
print(geo_1.head(10))
print(geo_2.head(10))

      id        f0        f1        f2     product
0  txEyH  0.705745 -0.497823  1.221170  105.280062
1  2acmU  1.334711 -0.340164  4.365080   73.037750
2  409Wp  1.022732  0.151990  1.419926   85.265647
3  iJLyR -0.032172  0.139033  2.978566  168.620776
4  Xdl7t  1.988431  0.155413  4.751769  154.036647
5  wX4Hy  0.969570  0.489775 -0.735383   64.741541
6  tL6pL  0.645075  0.530656  1.780266   49.055285
7  BYPU6 -0.400648  0.808337 -5.624670   72.943292
8  j9Oui  0.643105 -0.551583  2.372141  113.356160
9  OLuZU  2.173381  0.563698  9.441852  127.910945
      id         f0         f1        f2     product
0  kBEdx -15.001348  -8.276000 -0.005876    3.179103
1  62mP7  14.272088  -3.475083  0.999183   26.953261
2  vyE1P   6.263187  -5.948386  5.001160  134.766305
3  KcrkZ -13.081196 -11.506057  4.999415  137.945408
4  AHL4O  12.702195  -8.147433  5.004363  134.766305
5  HHckp  -3.327590  -2.205276  3.003647   84.038886
6  h5Ujo -11.142655 -10.133399  4.002382  110.992147
7  muH9x   4.23

In [6]:
geo_0.info()
geo_1.info()
geo_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null 

Во всех трех датасетах все данные заполнены корректно, без пропусков.

Посмотрим на разброс данных:

In [5]:
print('min geo_0 \n',geo_0[['f0','f1','f2']].min())
print('min geo_1 \n',geo_1[['f0','f1','f2']].min())
print('min geo_2 \n',geo_2[['f0','f1','f2']].min())

print('max geo_0 \n',geo_0[['f0','f1','f2']].max())
print('max geo_1 \n',geo_1[['f0','f1','f2']].max())
print('max geo_2 \n',geo_2[['f0','f1','f2']].max())

min geo_0 
 f0    -1.408605
f1    -0.848218
f2   -12.088328
dtype: float64
min geo_1 
 f0   -31.609576
f1   -26.358598
f2    -0.018144
dtype: float64
min geo_2 
 f0    -8.760004
f1    -7.084020
f2   -11.970335
dtype: float64
max geo_0 
 f0     2.362331
f1     1.343769
f2    16.003790
dtype: float64
max geo_1 
 f0    29.421755
f1    18.734063
f2     5.019721
dtype: float64
max geo_2 
 f0     7.238262
f1     7.844801
f2    16.739402
dtype: float64


Во всех трех датасетах разброс значений по всем трем признакам находятся в приблизительно похожих диапазонах - следовательно масштабирование не требуется.

Признак id не несет никакой полезной информации для обучения моделей. Удалим его:

In [6]:
geo_0.drop('id', axis=1, inplace=True)
geo_1.drop('id', axis=1, inplace=True)
geo_2.drop('id', axis=1, inplace=True)

## Обучение и проверка модели

Разобьем данные на тестовые и валидационные выборки:

In [7]:
target_0 = geo_0['product']
features_0 = geo_0.drop('product',axis=1)

features_train_0, features_valid_0, target_train_0, target_valid_0 = train_test_split(
    features_0, target_0, test_size=0.25, random_state=123)

print('Тренировочная выборка 0:',features_train_0.shape, target_train_0.shape)
print('Валидационная выборка 0:',features_valid_0.shape, target_valid_0.shape)

target_1 = geo_1['product']
features_1 = geo_1.drop('product',axis=1)

features_train_1, features_valid_1, target_train_1, target_valid_1 = train_test_split(
    features_1, target_1, test_size=0.25, random_state=123)

print('Тренировочная выборка 1:',features_train_1.shape, target_train_1.shape)
print('Валидационная выборка 1:',features_valid_1.shape, target_valid_1.shape)

target_2 = geo_2['product']
features_2 = geo_2.drop('product',axis=1)

features_train_2, features_valid_2, target_train_2, target_valid_2 = train_test_split(
    features_2, target_2, test_size=0.25, random_state=123)

print('Тренировочная выборка 2:',features_train_2.shape, target_train_2.shape)
print('Валидационная выборка 2:',features_valid_2.shape, target_valid_2.shape)

Тренировочная выборка 0: (75000, 3) (75000,)
Валидационная выборка 0: (25000, 3) (25000,)
Тренировочная выборка 1: (75000, 3) (75000,)
Валидационная выборка 1: (25000, 3) (25000,)
Тренировочная выборка 2: (75000, 3) (75000,)
Валидационная выборка 2: (25000, 3) (25000,)


In [8]:
model_0 = LinearRegression()
model_0.fit(features_train_0,target_train_0)
predictions_valid_0 = model_0.predict(features_valid_0)
rmse_0 = mean_squared_error(target_valid_0,predictions_valid_0) ** 0.5
print('rmse 0:',rmse_0)
print('Средний предсказанный объем:',predictions_valid_0.mean())


rmse 0: 37.64786282376177
Средний предсказанный объем: 92.54936189116306


In [9]:
model_1 = LinearRegression()
model_1.fit(features_train_1,target_train_1)
predictions_valid_1 = model_1.predict(features_valid_1)
rmse_1 = mean_squared_error(target_valid_1,predictions_valid_1) ** 0.5
print('rmse 1:',rmse_1)
print('Средний предсказанный объем:',predictions_valid_1.mean())


rmse 1: 0.8954139804944313
Средний предсказанный объем: 69.28001860653976


In [10]:
model_2 = LinearRegression()
model_2.fit(features_train_2,target_train_2)
predictions_valid_2 = model_2.predict(features_valid_2)
rmse_2 = mean_squared_error(target_valid_2,predictions_valid_2) ** 0.5
print('rmse 2:',rmse_2)
print('Средний предсказанный объем:',predictions_valid_2.mean())


rmse 2: 40.12803006598514
Средний предсказанный объем: 95.09859933591373


 - Средние объемы сырья на одну скважину в регионах №0 и №2 значительно больше чем в регионе №1. 
 
 - Так же важно отметить, что метрика качества RMSE в регионах №0 и №2 значительно хуже чем в регионе №1. Т.е. точность предсказаний модели для этого региона значительно выше.

 - Однако это еще не означает, что эти регионы больше подходят для разработки чем №1. Потому что разрабатываться будут всего 200 скважин из 100 000. 

 - Если в одном регионе распредление сырья по скважинам неравномерное, то может так оказаться что лучшие скважины в суммарно (по всем скважинам) более плохом регионе прибыльнее чем лучшие скважины в других регионах. 

## Подготовка к расчёту прибыли

In [11]:
# Бюджет на разработку 200 скважин:
BUDGET  = 10**10

# Прибыль от добычи одной тысячи баррелей:
PRICE = 4.5*10**5

# Допустимая вероятность убытка:
LOSS_PROB = 0.025

# Количество скважин для исследования:
WELLS = 500

# Количество скважин для разработки:
BEST_WELLS = 200

# Достаточный объем сырья для безубыточной разработки скважины: 
enough_volume = BUDGET / 200 / PRICE

print('Достаточный объем сырья для безубыточной разработки скважины:', round(enough_volume,2))

Достаточный объем сырья для безубыточной разработки скважины: 111.11


Средние объемы добычи сырья на одной скважине в регионах №0, №1 и №2, соответственно: 93, 69, 95. Видим что это значительно меньше уровня безубыточности. Т.е. экономически целесообразно разрабатывать только лучшие скважины в любом из регионов. 

## Расчёт прибыли и рисков 

Функция для расчета прибыли, с учетом условий задачи:

 - из 500 скважин для разработки выбираем 200 лучших скважин по предсказаниям модели.
  
 - заданный бюджет на разработку 200 скважин.
 
Для выбранных 200 скважин посчитаем объем добычи по целевому признаку и прибыль с учетом фиксированной цены.

In [12]:
def get_profit(target,predictions,sample_size,top_size,price,budget,state):
    predict_sample = predictions.sample(n=sample_size, replace=True, random_state=state)
    predict_top = predict_sample.sort_values(ascending=False)[:top_size]
    target_top = target.iloc[predict_top.index]
    volume = sum(target_top)
    profit = volume * price - budget
    return profit   

Для каждого региона оценим среднюю прибыль (для лучших скважин), а так же риски убытков и доверительный интервал. Для этого построим распределение прибыли по 1000 случайных выборок. Будем использовать методику bootstrap: 

In [13]:
def bootstraper(target,predictions,sample_size,top_size,price,budget,num_of_iterations):
    state = np.random.RandomState(123)
    profits = []
    for i in range(num_of_iterations):
        profit = get_profit(target,predictions,sample_size,top_size,price,budget,state)
        profits.append(profit)
    return pd.Series(profits)

In [14]:
predictions_valid_0 = pd.Series(predictions_valid_0)
predictions_valid_1 = pd.Series(predictions_valid_1)
predictions_valid_2 = pd.Series(predictions_valid_2)

target_valid_0 = target_valid_0.reset_index(drop=True)
target_valid_1 = target_valid_1.reset_index(drop=True)
target_valid_2 = target_valid_2.reset_index(drop=True)

In [15]:
num_of_iterations = 1000

profits_0 = bootstraper(target_valid_0,predictions_valid_0,WELLS,BEST_WELLS,PRICE,BUDGET,num_of_iterations)
profits_1 = bootstraper(target_valid_1,predictions_valid_1,WELLS,BEST_WELLS,PRICE,BUDGET,num_of_iterations)
profits_2 = bootstraper(target_valid_2,predictions_valid_2,WELLS,BEST_WELLS,PRICE,BUDGET,num_of_iterations)

### Средняя прибыль

In [16]:
print("Cредняя суммарная прибыль от 200 лучших скважин в регионе №0: {:.2f}".format(profits_0.mean()))
print("Cредняя суммарная прибыль от 200 лучших скважин в регионе №1: {:.2f}".format(profits_1.mean()))
print("Cредняя суммарная прибыль от 200 лучших скважин в регионе №2: {:.2f}".format(profits_2.mean()))

Cредняя суммарная прибыль от 200 лучших скважин в регионе №0: 477416824.27
Cредняя суммарная прибыль от 200 лучших скважин в регионе №1: 479190161.30
Cредняя суммарная прибыль от 200 лучших скважин в регионе №2: 343454376.58


 - наибольшая средняя прибыль для 200 лучших скважин в регионе №1.

### Доверительные интервалы:

In [17]:
alpha = 0.05

lower_limit_0 = profits_0.quantile(alpha/2)
upper_limit_0 = profits_0.quantile(1-alpha/2)
print('95%-ый доверительный интервал, регион №0: от {:.0f} до {:.0f} тыс.руб.'.format(lower_limit_0/1000, upper_limit_0/1000))

95%-ый доверительный интервал, регион №0: от -57994 до 974822 тыс.руб.


In [18]:
lower_limit_1 = profits_1.quantile(alpha/2)
upper_limit_1 = profits_1.quantile(1-alpha/2)
print('95%-ый доверительный интервал, регион №1: от {:.0f} до {:.0f} тыс.руб.'.format(lower_limit_1/1000, upper_limit_1/1000))

95%-ый доверительный интервал, регион №1: от 58727 до 874425 тыс.руб.


In [19]:
lower_limit_2 = profits_2.quantile(alpha/2)
upper_limit_2 = profits_2.quantile(1-alpha/2)
print('95%-ый доверительный интервал, регион №2: от {:.0f} до {:.0f} тыс.руб.'.format(lower_limit_2/1000, upper_limit_2/1000))

95%-ый доверительный интервал, регион №2: от -231376 до 860841 тыс.руб.


 - из всех трех регионов только у региона №1 нижняя граница 95% доверительного интервала не заходит в зону убытков.

### Вероятность убытков:

In [20]:
loss_prob = profits_0[profits_0<0].count()/profits_0.count()
print('Вероятность убытка в регионе №0: {:.2%}'.format(loss_prob))
loss_prob = profits_1[profits_1<0].count()/profits_1.count()
print('Вероятность убытка в регионе №1: {:.2%}'.format(loss_prob))
loss_prob = profits_2[profits_2<0].count()/profits_2.count()
print('Вероятность убытка в регионе №2: {:.2%}'.format(loss_prob))

Вероятность убытка в регионе №0: 4.10%
Вероятность убытка в регионе №1: 0.90%
Вероятность убытка в регионе №2: 9.90%


 - критериям задачи (максимальный риск убытка не должен превышать 2.5%) соответствует только регион №1.

## Выводы:

 - метрика качества RMSE показывает что наиболее точные предсказания дает модель обученная на данных в регионе №1

 - Наибольшую среднюю прибыль при разработке 200 лучших скважин даст второй регион (несмотря на то что средний объем сырья на одну скважину по всему датасету для этого региона самый маленький - в этом регионе нефть распределена не так равномерно, как в первом и третьем регионах). 
 
 - 95% доверительный интервал не заходит в зону убытков только в регионе №1

 - Из всех трех регионов вероятность убытка удовлетворяет критериям задачи (не более 2.5%) только регион №1

 - Следовательно можно рекомендовать к разработке регион №1. 